In [1]:
from torchrecsys.datasets import InteractionsDataset, SequenceDataset
from torchrecsys.models import BaseModel, NeuralCF
from torchrecsys.task import Ranking
from torchrecsys.layers import BruteForceLayer
from torchrecsys.utils import feature_catalog
from torchrecsys import Trainer
import pandas as pd

from torch.utils.data import DataLoader

In [2]:
ratings = pd.read_csv("ml-100k/ratings.csv").rename(columns={"userId": "user", "movieId": "item"})
movies = pd.read_csv("ml-100k/movies.csv").rename(columns={"movieId": "item"})
users = ratings[["user"]].drop_duplicates()


print(movies)
movies['title'], uniques = pd.factorize(movies['title'])
movies['genres'], uniques = pd.factorize(movies['genres'])

movies['title'] = movies.title.astype('category')
movies['genres'] = movies.genres.astype('category')

        item                                      title  \
0          1                           Toy Story (1995)   
1          2                             Jumanji (1995)   
2          3                    Grumpier Old Men (1995)   
3          4                   Waiting to Exhale (1995)   
4          5         Father of the Bride Part II (1995)   
...      ...                                        ...   
9737  193581  Black Butler: Book of the Atlantic (2017)   
9738  193583               No Game No Life: Zero (2017)   
9739  193585                               Flint (2017)   
9740  193587        Bungo Stray Dogs: Dead Apple (2018)   
9741  193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                              

In [3]:
dataset = InteractionsDataset(ratings, users, movies, target_column=True)
train = DataLoader(dataset, batch_size=5)

In [4]:
for u in train:
    break

In [5]:
dataset.data_schema

{'interactions': [610, 193609],
 'context': [],
 'user_features': [['int64', 610]],
 'item_features': [['int64', 9742], ['category', 9737], ['category', 951]],
 'objetive': 'notbinary?'}

In [6]:
dataset[0]

(array([1., 1., 4.]), array([], dtype=float64), array([1]), array([1, 0, 0]))

In [7]:

model = NeuralCF(dataset.data_schema)

In [8]:
u

[tensor([[ 1.,  1.,  4.],
         [ 1.,  3.,  4.],
         [ 1.,  6.,  4.],
         [ 1., 47.,  5.],
         [ 1., 50.,  5.]], dtype=torch.float64),
 tensor([], size=(5, 0), dtype=torch.float64),
 tensor([[1],
         [1],
         [1],
         [1],
         [1]]),
 tensor([[ 1,  0,  0],
         [ 3,  2,  2],
         [ 6,  5,  5],
         [47, 43, 27],
         [50, 46, 29]])]

In [9]:
u[0][:, 0]

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

In [10]:
model.criterion

MSELoss()

In [11]:
model.n_items

193609

In [12]:
movies.item.max()

193609

In [ ]:
trainer = Trainer()
trainer.fit(model, train)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/jafen/miniconda3/envs/hm37/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name               | Type      | Params
-------------------------------------------------
0 | user_embedding     | Embedding | 39.1 K
1 | item_embedding     | Embedding | 12.4 M
2 | linear             | Linear    | 33.3 K
3 | final_linear       | Linear    | 513   
4 | feature1_embedding | Embedding | 77.9 K
5 | feature2_embedding | Embedding | 7.6 K 
6 | criterion          | MSELoss   | 0     
-------------------------------------------------
12.5 M    Trainable params
0         Non-trainable params
12.5 M    Total params
50.198    Total estimated model params s

Training: 0it [00:00, ?it/s]

In [ ]:
import torch
x = torch.zeros(5, 1)
torch.squeeze(x).shape